# Training Data

### Labeling DataSet

In [28]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report 
import numpy as np
import sklearn
import os 
import shutil 
import random

In [3]:
train_gen = ImageDataGenerator(rescale=1./255)

# Turning every folder name into class label
train_data = train_gen.flow_from_directory(
    "../Datasets/combined-cleaned-dataset",
    target_size=(224,224),
    batch_size=32,
    class_mode= "categorical"
)

print(train_data.class_indices)

Found 18042 images belonging to 13 classes.
{'battery': 0, 'biological': 1, 'brown-glass': 2, 'cardboard': 3, 'clothes': 4, 'glass': 5, 'green-glass': 6, 'metal': 7, 'paper': 8, 'plastic': 9, 'shoes': 10, 'trash': 11, 'white-glass': 12}


In [8]:
# Spliting our dataset into training, validation, and testing 
def split_dataset(source_dir, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    random.seed(42)
    
    for class_folder in os.listdir(source_dir):
        class_path = os.path.join(source_dir, class_folder)
        if not os.path.isdir(class_path):
            continue 
        
        images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg','.jpeg','.png'))]
        random.shuffle(images)
    
        total = len(images)
        train_end = int(total * train_ratio)
        val_end = train_end + int(total * val_ratio)
    
        train_images = images[:train_end]
        val_images = images[train_end:val_end]
        test_images = images[val_end:]
    
        train_class_dir = os.path.join(output_dir, 'train', class_folder)
        val_class_dir = os.path.join(output_dir, 'val', class_folder)
        test_class_dir = os.path.join(output_dir,'test',class_folder)
    
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(val_class_dir, exist_ok=True)
        os.makedirs(test_class_dir, exist_ok=True)
    
        for img in train_images:
            shutil.copy2(os.path.join(class_path, img), os.path.join(train_class_dir, img))

        for img in val_images:
            shutil.copy2(os.path.join(class_path, img), os.path.join(val_class_dir, img))

        for img in test_images:
            shutil.copy2(os.path.join(class_path, img), os.path.join(test_class_dir, img))

        print(f" {class_folder}: {len(train_images)} train / {len(val_images)} val / {len(test_images)} test")
    
if __name__ == "__main__":
    source = "../Datasets/combined-cleaned-dataset"   
    destination = "../Notebooks/the_final_sortdown"          
    split_dataset(source, destination, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)
    

 paper: 1150 train / 246 val / 248 test
 green-glass: 440 train / 94 val / 95 test
 clothes: 3727 train / 798 val / 800 test
 metal: 825 train / 176 val / 178 test
 cardboard: 905 train / 194 val / 195 test
 trash: 583 train / 125 val / 126 test
 glass: 350 train / 75 val / 76 test
 biological: 689 train / 147 val / 149 test
 white-glass: 542 train / 116 val / 117 test
 battery: 661 train / 141 val / 143 test
 brown-glass: 424 train / 91 val / 92 test
 plastic: 942 train / 202 val / 203 test
 shoes: 1383 train / 296 val / 298 test


In [19]:
# Data Augmentation
train_gen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    brightness_range = [0.8, 1.2]
)

print("Training Data:")
train_data = train_gen.flow_from_directory(
    "the_final_sortdown/train",
    batch_size=32,
    target_size=(224, 224),
    class_mode = "categorical"
)

val_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

print("\nValidation Data:")
val_data = val_gen.flow_from_directory(
    "the_final_sortdown/val",
    batch_size=32,
    target_size=(224, 224),
    class_mode="categorical"
)

print("\nTesting Data:")
test_data = test_gen.flow_from_directory(
    "the_final_sortdown/test",
    batch_size=32,
    target_size=(224, 224),
    class_mode="categorical",
    shuffle=False
)

Training Data:
Found 12621 images belonging to 13 classes.

Validation Data:
Found 2701 images belonging to 13 classes.

Testing Data:
Found 2720 images belonging to 13 classes.


In [31]:
# Transfer Learning MobileNetV2
base_model = MobileNetV2(
    include_top = False,
    weights ='imagenet',
    input_shape = (224,224,3)
)
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(train_data.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=Adam(),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

#model.summary()

In [21]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs = 10
)

/Users/cindychen/.pyenv/versions/3.9.16/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 86s 215ms/step - accuracy: 0.7423 - loss: 0.8368 - val_accuracy: 0.8837 - val_loss: 0.3776
Epoch 2/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 86s 217ms/step - accuracy: 0.8853 - loss: 0.3429 - val_accuracy: 0.8849 - val_loss: 0.3476
Epoch 3/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 85s 216ms/step - accuracy: 0.9005 - loss: 0.2749 - val_accuracy: 0.8923 - val_loss: 0.2966
Epoch 4/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 85s 215ms/step - accuracy: 0.9221 - loss: 0.2305 - val_accuracy: 0.8982 - val_loss: 0.2971
Epoch 5/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 81s 204ms/step - accuracy: 0.9295 - loss: 0.1896 - val_accuracy: 0.8971 - val_loss: 0.3186
Epoch 6/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 86s 216ms/step - accuracy: 0.9322 - loss: 0.1862 - val_accuracy: 0.9093 - val_loss: 0.2801
Epoch 7/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 82s 206ms/step - accuracy: 0.9373 - loss: 0.1621 - val_accuracy: 0.9067 - val_loss: 0.2806
Epoch 8/10
395/395 ━━━━━━━━━━━━━━━━━━━━ 83s 209ms/step - accuracy: 0.9438 - loss: 0

In [30]:
test_loss, test_acc = model.evaluate(test_data)
print(f"Test accuarcy: {test_acc:.2f}")

85/85 ━━━━━━━━━━━━━━━━━━━━ 13s 151ms/step - accuracy: 0.9400 - loss: 0.2037
Test accuarcy: 0.91


In [29]:
# Adding Precision, Recall, and F1
y_pred = model.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)

y_true = test_data.classes
print(classification_report(y_true, y_pred_classes, target_names=list(test_data.class_indices.keys())))


85/85 ━━━━━━━━━━━━━━━━━━━━ 13s 153ms/step
              precision    recall  f1-score   support

     battery       0.94      0.94      0.94       143
  biological       0.95      0.98      0.96       149
 brown-glass       0.86      0.83      0.84        92
   cardboard       0.88      0.97      0.92       195
     clothes       0.99      0.98      0.99       800
       glass       0.38      0.39      0.39        76
 green-glass       0.79      0.84      0.82        95
       metal       0.80      0.92      0.85       178
       paper       0.95      0.94      0.94       248
     plastic       0.95      0.81      0.87       203
       shoes       0.94      0.99      0.96       298
       trash       0.95      0.83      0.89       126
 white-glass       0.70      0.61      0.65       117

    accuracy                           0.91      2720
   macro avg       0.85      0.85      0.85      2720
weighted avg       0.91      0.91      0.91      2720



In [ ]:
# Adding Reinforcement Learning Sysytem
